In [125]:
import pandas as pd
import numpy as np
import os
import os.path as path

In [126]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"

In [127]:
# csv_name_list = os.listdir(data_path)

In [128]:
# print(csv_name_list)

In [129]:
# for csv_name in csv_name_list:
#     print("===================={}===================".format(csv_name))
#     df = pd.read_csv(path.join(data_path, csv_name))
#     print(df.describe())

In [130]:
def print_isna(df):
    print(pd.isna(df).sum())

# 학습 데이터 셋 만들기

# INPUT

> 소득
    - INC || INCOME, HOUSE_INCOME <========== tn_traveller_master_여행객 Master_C 

    특이사항 - 소득 값이 두가지 존재

    INCOMDE사용으로 일단 대체
    
> 미션
    - MIS || TRAVEL_MISSION(==TRAVEL_PURPOSE) <========== tn_travel_여행_C

    특이사항 - 여행 미션내 값들이 여러개 존재

> 여행동반자관계
    - TCR || REL_CD <========== tn_companion_info_동반자정보_C

> 숙소유형 
    - HTY || LODGING_TYPE_CD <========= tn_visit_area_info_방문지정보_C


> 성별
    - GEN || GENDER  <========== tn_traveller_master_여행객 Master_C

    특이사항 남, 여로 적혀 있음

    해당 갑은 남 : 1, 여 : 0으로 변경

> 여행동기
    - TMT || TRAVEL_MOTiVE_1 ~ 3 <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~3번 존재

    3가지 중복 없이 입력

> 여행스타일
    - STY || TRAVEL_STYL_1 ~ 8  <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~8번이 존재

    => 1개만 받을 예정
    학습 시 max값으로 대체

> 동반자연령대
    - AGE || COMPANION_AGE_GRP <============ tn_companion_info_동반자정보_C
    

#################################### 여기 까지는 후에 타 서버에서 받아서 예측시 입력해 줄 데이터 #####################################



#################################################################################################################################

# OUTPUT
 

> (DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~

> 여행지 만족도

    train 간에는 모델은 입력된 여행 데이터를 바탕으로 각 여행지의 점수를 맞추는 작업을 수행한다.
        
        출력한 점수 중 상위 k개의 장소를 선정 해당 장소 중에 정답이 존재한다면
    
    predict의 경우 모든 여행지에 대해서 점수를 매기는 작업 수행 -> softmax를 통해서 표준화하고 내림차순 정렬



> 여행 ID  ======>>>>>> 필요 X
    - || TRAVEL_ID <======= tn_traveller_master_여행객 Master_C

> 방문지 주소 # 전처리 후 리스트해서 전달
    - - || LOTNO_ADDR <====== tn_visit_area_info_방문지정보_C

    몇 항목만 사용
        VIS
            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            13: 체험 활동 관광지

            보류
                10:상점
                11:식당/카페
                12:기타
                24:숙소

            제거
                21:집
                22:친구/친구집
                23:사무실
                

2가지 경우가 존재

1. travel id를 확률을 맞추게하고

    2. 확률 분포를 이용 해당 분포 순으로 해당 id가 방문한 지역 전달

2. 방문지 수에 맞게 데이터를 확장한다.

    2. 이 경우 추가적인 데이터를 심어주는게 좋을 듯하다 예를 들어 순서 방문 순서 정보나 재방문 점수 같은 정보를 추가해서 유저 정보 + 다른 추가 정보 이용 학습


In [131]:
input_data_csv_name_list = ["tn_traveller_master_여행객 Master_C", "tn_travel_여행_C", "tn_companion_info_동반자정보_C", "tn_visit_area_info_방문지정보_C"]

In [132]:
output_data_csv_name_list = ["tn_travel_여행_C","tn_visit_area_info_방문지정보_C"]

In [133]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"
extender = '.csv'

index_dict = {'TRAVEL_ID':['1']}
# input_dict = {'여행객':[],'여행':[],'동반자정보':[],'방문지정보':[]}
# 방문지 정보는 output에서 같이 처리
input_dict = {'여행객':[],'여행':[],'동반자정보':[]}
output_dict = {'TRAVELER_ID':[],'방문지정보':[]}
print(index_dict)
print(input_dict)
print(output_dict)
print(data_path)

{'TRAVEL_ID': ['1']}
{'여행객': [], '여행': [], '동반자정보': []}
{'TRAVELER_ID': [], '방문지정보': []}
./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv


In [134]:
tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))

index_list = tmp_df['TRAVELER_ID'].values
index_dict['TRAVEL_ID'] = index_list
print(index_dict)
# print(tmp_df['TRAVELER_ID'].values)

{'TRAVEL_ID': array(['c003867', 'c000679', 'c013674', ..., 'c008277', 'c012053',
       'c007519'], dtype=object)}


In [135]:
get_list = ['TRAVELER_ID','INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1','TRAVEL_MOTIVE_2','TRAVEL_MOTIVE_3', 
            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))
tmp_value = tmp_df['TRAVELER_ID'].values

tmp_value = [ s[0]+"_"+s for s in tmp_value ]
tmp_df['TRAVEL_ID'] = tmp_value
tmp_df = tmp_df.drop('TRAVELER_ID', axis='columns')

tmp_df = tmp_df.set_index("TRAVEL_ID")

input_dict['여행객'] = tmp_df[get_list[1:]]
print(input_dict['여행객'].columns)
print(input_dict['여행객'].head())


Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
           INCOME  HOUSE_INCOME  TRAVEL_MOTIVE_1  TRAVEL_MOTIVE_2  \
TRAVEL_ID                                                           
c_c003867       4           NaN              1.0              7.0   
c_c000679       4           NaN              5.0              3.0   
c_c013674       4           6.0              1.0              8.0   
c_c010826       1           1.0              7.0              2.0   
c_c012020       3           7.0              3.0              1.0   

           TRAVEL_MOTIVE_3  TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  \
TRAVEL_ID                                                                 
c_c003867              2.0              2              1              2   
c_c000679            

In [136]:
get_list = ['TRAVEL_ID', 'TRAVEL_MISSION']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])

input_dict['여행'] = tmp_df[get_list[1:]]
print_isna(input_dict['여행'])

TRAVEL_MISSION    0
dtype: int64


In [137]:
get_list = ['TRAVEL_ID', 'REL_CD', 'COMPANION_AGE_GRP']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[2]+extender))
tmp_df = tmp_df.set_index(get_list[0])
input_dict['동반자정보'] = tmp_df[get_list[1:]]
print_isna(input_dict['동반자정보'])

REL_CD               0
COMPANION_AGE_GRP    0
dtype: int64


In [138]:
# get_list = ['TRAVEL_ID', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD']

# tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[3]+extender))
# tmp_df = tmp_df.set_index(get_list[0])
# input_dict['방문지정보'] = tmp_df[get_list[1:]]
# # print(tmp_df)

"""output_dict의 방문지정보에서 같이 처리"""

'output_dict의 방문지정보에서 같이 처리'

In [139]:
get_list = ['TRAVEL_ID','TRAVELER_ID']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[0]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['TRAVELER_ID'] = tmp_df[get_list[1:]]
# print(output_dict['TRAVELER_ID'])
print_isna(output_dict['TRAVELER_ID'])

TRAVELER_ID    0
dtype: int64


In [140]:
get_list = ['TRAVEL_ID', 'VISIT_AREA_NM','LOTNO_ADDR','X_COORD','Y_COORD', 'VISIT_AREA_TYPE_CD']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['방문지정보'] = tmp_df[get_list[1:]]
print_isna(output_dict['방문지정보'])
# print(tmp_df)

VISIT_AREA_NM            0
LOTNO_ADDR            8381
X_COORD               7330
Y_COORD               7330
VISIT_AREA_TYPE_CD       0
dtype: int64


C:\Users\jinmi\AppData\Local\Temp\ipykernel_18544\2150674066.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))


In [141]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.index)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['c_c003867', 'c_c000679', 'c_c013674', 'c_c010826', 'c_c012020',
       'c_c009357', 'c_c012760', 'c_c003756', 'c_c003541', 'c_c002909',
       ...
       'c_c013812', 'c_c012432', 'c_c009023', 'c_c010863', 'c_c001641',
       'c_c005317', 'c_c000381', 'c_c008277', 'c_c012053', 'c_c007519'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['c_c011995', 'c_c008882', 'c_c003940', 'c_c012638', 'c_c011983',
       'c_c012367', 'c_c013909', 'b_b006673', 'c_c012881', 'c_c002721',
       ...
       'c_c002389', 'c_c003692', 'c_c013085', 'c_c011684', 'c_c000183',
       'c_c012049', 'c_c010871', 'c_c004312', 'c_c009302', 'c_c000104'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
동반자정보
-------------------------------------

In [142]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
for keys in output_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = output_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
TRAVELER_ID

# 방분지 정보 정제

> 아래는 해당 번호들 제외하고 걸러야한다

VIS - VISIT_AREA_TYPE_CD 사용할 항목들

            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            11: 식당 카페
            13: 체험 활동 관광지

            


> 아래는 계산해서 SCORE로 추가

(DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~5


> 최종 11352개의 방문지 존재 해당 정보를 바탕으로 데이터 수립 

    TRAVERL_ID에 해당 하는 정보들을 추가 적으로 붙여넣어 준다.



In [143]:
tmp_df = output_dict['방문지정보']
print(tmp_df.head())

new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,11,13])]
# new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)
# print(new_df)
output_dict['방문지정보'] = new_df
# print(output_dict['방문지정보'])

duplicated_index = new_df.index.duplicated()
print(duplicated_index)
print(new_df.columns)
print(pd.unique(new_df['VISIT_AREA_TYPE_CD']))
print(pd.unique(new_df['LOTNO_ADDR']))

          VISIT_AREA_NM             LOTNO_ADDR     X_COORD    Y_COORD  \
TRAVEL_ID                                                               
c_c005733       3000냥밥집  충북 영동군 영동읍 계산리 653-12  127.785733   36.17142   
c_c000326    소노캄여수 마레첼로         전남 여수시 수정동 772  127.754615  34.742572   
c_c003528      생거진천문화축제     충북 진천군 진천읍 읍내리 165  127.446235  36.858789   
c_c001050           오목대    전북 전주시 완산구 교동 산 1-3  127.154583  35.814382   
c_c001776       남당노을전망대  충남 홍성군 서부면 남당리 767-18  126.465582  36.544343   

           VISIT_AREA_TYPE_CD  
TRAVEL_ID                      
c_c005733                  11  
c_c000326                  11  
c_c003528                   8  
c_c001050                   2  
c_c001776                   1  
[False False False ...  True  True  True]
Index(['VISIT_AREA_NM', 'LOTNO_ADDR', 'X_COORD', 'Y_COORD',
       'VISIT_AREA_TYPE_CD'],
      dtype='object')
[11  8  2  1  6 13  7  4  3  5]
['충북 영동군 영동읍 계산리 653-12' '전남 여수시 수정동 772' '충북 진천군 진천읍 읍내리 165' ...
 '전남 화순군 능주

In [144]:
final_df = new_df

In [145]:
target_col = "LOTNO_ADDR"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])

(19964, 5)
1604
(18360, 5)
0


In [146]:
final_df = tmp_df

In [147]:
target_col = "X_COORD"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18360, 5)
24
(18336, 5)
0


In [148]:
print(pd.isna(tmp_df).sum())

VISIT_AREA_NM         0
LOTNO_ADDR            0
X_COORD               0
Y_COORD               0
VISIT_AREA_TYPE_CD    0
dtype: int64


In [149]:
tmp_df = tmp_df.reset_index()

In [150]:
tmp_df = tmp_df.drop("TRAVEL_ID", axis=1)

In [151]:
tmp_df

,VISIT_AREA_NM,LOTNO_ADDR,X_COORD,Y_COORD,VISIT_AREA_TYPE_CD
0,3000냥밥집,충북 영동군 영동읍 계산리 653-12,127.785733,36.17142,11
1,소노캄여수 마레첼로,전남 여수시 수정동 772,127.754615,34.742572,11
2,생거진천문화축제,충북 진천군 진천읍 읍내리 165,127.446235,36.858789,8
3,오목대,전북 전주시 완산구 교동 산 1-3,127.154583,35.814382,2
4,남당노을전망대,충남 홍성군 서부면 남당리 767-18,126.465582,36.544343,1
...,...,...,...,...,...
18331,제이에스빌,전북 군산시 조촌동 922-14,126.741422,35.96917085,11
18332,초원사진관,전북 군산시 신창동 1-5,126.708372,35.98773979,2
18333,성심당 본점,대전 중구 은행동 145-1,127.427298,36.32766877,11
18334,국립공주박물관,충남 공주시 웅진동 360,127.111970,36.46550084,2


In [152]:
tmp_df.to_csv("./data_visit_for_db.csv")